In [1]:
import pandas as pd
import numpy as np

In [7]:
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
from mxnet import  autograd,nd

In [76]:
train_set = pd.read_csv("Processed_train.csv")
sub = pd.read_csv("Processed_test.csv")

In [77]:
X = train_set.iloc[: ,1:]
y = train_set["Survived"]


In [84]:
X = X.values
y = y.values

In [85]:
X

array([[3, 1, 1, ..., 2, 0, 1],
       [1, 0, 2, ..., 2, 0, 3],
       [3, 0, 1, ..., 1, 1, 2],
       ..., 
       [3, 0, 1, ..., 4, 0, 2],
       [1, 1, 1, ..., 1, 1, 1],
       [3, 1, 1, ..., 1, 1, 1]], dtype=int64)

In [86]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [87]:
scaler = StandardScaler()

In [88]:
scaler.fit(X_train)

c:\users\caiwei\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [89]:
X_train = scaler.transform(X_train)

c:\users\caiwei\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [90]:
X_test = scaler.transform(X_test)

c:\users\caiwei\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [91]:
sub = scaler.transform(sub)

In [92]:
X_train.shape

(712, 11)

In [96]:
X_train = X_train.astype("float32")
y_train = y_train.astype("float32")
X_test  = X_test.astype("float32")
y_test = y_test.astype("float32")
sub = sub.astype("float32")

In [97]:
from mxnet.gluon import  data as gdata

In [98]:
train_set = gdata.ArrayDataset(X_train,y_train)
test_set = gdata.ArrayDataset(X_test,y_test)

In [99]:
batch_size = 10
train_iter = gdata.DataLoader(train_set,batch_size,shuffle=True)
test_iter = gdata.DataLoader(test_set,batch_size,shuffle=True)

In [100]:
for X,y in train_iter:
    print(X,y)
    break


[[ 0.82389838  0.7333464   0.77569646 -0.47480127 -1.34116137 -0.58281463
  -0.98057449 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [ 0.82389838  0.7333464  -1.61519468  0.76350826 -0.44369248  0.98308736
   0.44113564 -0.54272044  0.06331877 -1.23483706  2.28917289]
 [ 0.82389838 -1.36361206 -0.41974914  2.00181794 -0.44369248 -0.58281463
   2.40965748 -0.54272044  0.672548   -1.23483706  1.2922976 ]
 [-0.37327641 -1.36361206 -0.41974914 -0.47480127 -0.44369248  0.98308736
   0.11304869 -0.54272044  0.06331877 -1.23483706  0.29542232]
 [ 0.82389838 -1.36361206 -0.41974914 -0.47480127 -1.34116137 -0.58281463
   0.44113564 -0.54272044 -0.54591042  0.80982339  0.29542232]
 [ 0.82389838  0.7333464  -0.41974914 -0.47480127 -1.34116137 -0.58281463
  -0.10567595 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [ 0.82389838  0.7333464  -0.41974914 -0.47480127 -0.44369248 -0.58281463
  -0.21503827 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [-0.37327641 -1.36361206  1.97114205 -0

In [101]:
from mxnet.gluon import  loss as gloss,nn
from mxnet import gluon,init

In [102]:
num_inputs = 11
num_outputs = 2
W = nd.random.normal(scale=0.01,shape=(num_inputs,num_outputs))
b = nd.zeros(num_outputs)

In [136]:
W.attach_grad()
b.attach_grad()

In [103]:
# 定义softmax()
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1,keepdims = True)
    return X_exp / partition

In [105]:
# 定义模型
def net(X):
    return softmax(nd.dot(X,W)+b)

In [129]:
for X,y in train_iter:
    print(X)
    print(net(X))
    print(cross_entropy(net(X),y))
    print(accuracy(net(X),y))
    break


[[ 0.82389838  0.7333464  -0.41974914 -0.47480127 -1.34116137 -0.58281463
  -1.08993685 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [ 0.82389838  0.7333464  -0.41974914 -0.47480127 -1.34116137  0.98308736
  -1.08993685 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [ 0.82389838 -1.36361206 -0.41974914 -0.47480127 -1.34116137  2.5489893
   0.22241101 -0.54272044 -0.54591042  0.80982339  0.29542232]
 [ 0.82389838  0.7333464  -0.41974914 -0.47480127  1.35124528 -0.58281463
  -1.63674843 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [-0.37327641  0.7333464   1.97114205  0.76350826  1.35124528 -0.58281463
   0.65986031 -0.54272044  0.672548   -1.23483706 -0.70145297]
 [-0.37327641  0.7333464   1.97114205 -0.47480127 -0.44369248 -0.58281463
  -0.10567595 -0.54272044 -0.54591042  0.80982339 -0.70145297]
 [-0.37327641 -1.36361206 -0.41974914 -0.47480127  0.45377639 -0.58281463
   2.5190196  -0.54272044  0.06331877 -1.23483706  1.2922976 ]
 [ 0.82389838  0.7333464  -0.41974914 -0.

In [127]:
# 定义损失函数
def cross_entropy(y_hat,y):
    return -nd.pick(y_hat,y).log()

In [110]:
y_hat = nd.array([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat.argmax(axis=1)


[ 2.  2.]
<NDArray 2 @cpu(0)>

In [111]:
# 计算分类准确率
def accuracy(y_hat,y):
    return (y_hat.argmax(axis=1)==y.astype("float32")).mean().asscalar()

In [112]:
def evaluate_accuracy(data_iter,net):
    acc = 0
    for X,y in data_iter:
        acc += accuracy(net(X),y)
    return acc/len(data_iter)

In [130]:
evaluate_accuracy(test_iter,net)

0.34135803166362977

In [135]:
batch_size

10

In [139]:
num_epochs, lr = 100, 0.001

# 本函数已保存在 gluonbook 包中方便以后使用。
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum = 0
        train_acc_sum = 0
        for X, y in train_iter:
#             print(X,y)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
#                 print(l)
            l.backward()
            if trainer is None:
                gb.sgd(params, lr, batch_size)
            else:
                trainer.step(batch_size)  # 下一节将用到。
            train_l_sum += l.mean().asscalar()
            train_acc_sum += accuracy(y_hat, y)
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / len(train_iter),
                 train_acc_sum / len(train_iter), test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs,
          batch_size, [W, b], lr)

epoch 1, loss 0.4302, train acc 0.812, test acc 0.860
epoch 2, loss 0.4354, train acc 0.808, test acc 0.860
epoch 3, loss 0.4315, train acc 0.810, test acc 0.860
epoch 4, loss 0.4431, train acc 0.807, test acc 0.861
epoch 5, loss 0.4337, train acc 0.811, test acc 0.861
epoch 6, loss 0.4351, train acc 0.807, test acc 0.861
epoch 7, loss 0.4363, train acc 0.811, test acc 0.860
epoch 8, loss 0.4353, train acc 0.806, test acc 0.861
epoch 9, loss 0.4375, train acc 0.806, test acc 0.860
epoch 10, loss 0.4466, train acc 0.803, test acc 0.861
epoch 11, loss 0.4315, train acc 0.811, test acc 0.860
epoch 12, loss 0.4389, train acc 0.806, test acc 0.860
epoch 13, loss 0.4332, train acc 0.808, test acc 0.859
epoch 14, loss 0.4298, train acc 0.811, test acc 0.860
epoch 15, loss 0.4343, train acc 0.806, test acc 0.860
epoch 16, loss 0.4416, train acc 0.799, test acc 0.860
epoch 17, loss 0.4302, train acc 0.810, test acc 0.858
epoch 18, loss 0.4328, train acc 0.808, test acc 0.860
epoch 19, loss 0.43

epoch 151, loss 0.4314, train acc 0.812, test acc 0.860
epoch 152, loss 0.4311, train acc 0.811, test acc 0.860
epoch 153, loss 0.4348, train acc 0.803, test acc 0.860
epoch 154, loss 0.4346, train acc 0.807, test acc 0.860
epoch 155, loss 0.4346, train acc 0.811, test acc 0.860
epoch 156, loss 0.4328, train acc 0.811, test acc 0.860
epoch 157, loss 0.4320, train acc 0.811, test acc 0.860
epoch 158, loss 0.4306, train acc 0.810, test acc 0.860
epoch 159, loss 0.4311, train acc 0.812, test acc 0.861
epoch 160, loss 0.4309, train acc 0.812, test acc 0.860
epoch 161, loss 0.4322, train acc 0.811, test acc 0.860
epoch 162, loss 0.4300, train acc 0.811, test acc 0.859
epoch 163, loss 0.4315, train acc 0.810, test acc 0.860
epoch 164, loss 0.4297, train acc 0.811, test acc 0.860
epoch 165, loss 0.4320, train acc 0.814, test acc 0.861
epoch 166, loss 0.4313, train acc 0.810, test acc 0.861
epoch 167, loss 0.4303, train acc 0.810, test acc 0.860
epoch 168, loss 0.4314, train acc 0.808, test ac

epoch 298, loss 0.4306, train acc 0.811, test acc 0.861
epoch 299, loss 0.4394, train acc 0.800, test acc 0.860
epoch 300, loss 0.4330, train acc 0.810, test acc 0.861
epoch 301, loss 0.4404, train acc 0.804, test acc 0.861
epoch 302, loss 0.4299, train acc 0.808, test acc 0.860
epoch 303, loss 0.4308, train acc 0.808, test acc 0.860
epoch 304, loss 0.4390, train acc 0.808, test acc 0.860
epoch 305, loss 0.4423, train acc 0.806, test acc 0.859
epoch 306, loss 0.4310, train acc 0.810, test acc 0.860
epoch 307, loss 0.4361, train acc 0.803, test acc 0.859
epoch 308, loss 0.4322, train acc 0.808, test acc 0.860
epoch 309, loss 0.4301, train acc 0.812, test acc 0.860
epoch 310, loss 0.4309, train acc 0.812, test acc 0.860
epoch 311, loss 0.4312, train acc 0.810, test acc 0.861
epoch 312, loss 0.4297, train acc 0.812, test acc 0.860
epoch 313, loss 0.4356, train acc 0.808, test acc 0.860
epoch 314, loss 0.4325, train acc 0.814, test acc 0.859
epoch 315, loss 0.4349, train acc 0.804, test ac

epoch 446, loss 0.4303, train acc 0.810, test acc 0.860
epoch 447, loss 0.4305, train acc 0.811, test acc 0.860
epoch 448, loss 0.4350, train acc 0.811, test acc 0.860
epoch 449, loss 0.4305, train acc 0.811, test acc 0.860
epoch 450, loss 0.4295, train acc 0.815, test acc 0.860
epoch 451, loss 0.4297, train acc 0.808, test acc 0.860
epoch 452, loss 0.4312, train acc 0.808, test acc 0.860
epoch 453, loss 0.4321, train acc 0.808, test acc 0.861
epoch 454, loss 0.4318, train acc 0.808, test acc 0.861
epoch 455, loss 0.4383, train acc 0.807, test acc 0.861
epoch 456, loss 0.4295, train acc 0.811, test acc 0.860
epoch 457, loss 0.4315, train acc 0.808, test acc 0.859
epoch 458, loss 0.4310, train acc 0.812, test acc 0.860
epoch 459, loss 0.4325, train acc 0.808, test acc 0.861
epoch 460, loss 0.4399, train acc 0.807, test acc 0.860
epoch 461, loss 0.4303, train acc 0.814, test acc 0.860
epoch 462, loss 0.4319, train acc 0.811, test acc 0.860
epoch 463, loss 0.4394, train acc 0.797, test ac

epoch 594, loss 0.4319, train acc 0.810, test acc 0.861
epoch 595, loss 0.4307, train acc 0.811, test acc 0.860
epoch 596, loss 0.4327, train acc 0.808, test acc 0.861
epoch 597, loss 0.4332, train acc 0.811, test acc 0.860
epoch 598, loss 0.4417, train acc 0.801, test acc 0.861
epoch 599, loss 0.4302, train acc 0.811, test acc 0.861
epoch 600, loss 0.4295, train acc 0.812, test acc 0.860
epoch 601, loss 0.4453, train acc 0.806, test acc 0.861
epoch 602, loss 0.4301, train acc 0.811, test acc 0.859
epoch 603, loss 0.4299, train acc 0.811, test acc 0.867
epoch 604, loss 0.4408, train acc 0.804, test acc 0.860
epoch 605, loss 0.4318, train acc 0.811, test acc 0.859
epoch 606, loss 0.4304, train acc 0.811, test acc 0.859
epoch 607, loss 0.4309, train acc 0.811, test acc 0.860
epoch 608, loss 0.4314, train acc 0.814, test acc 0.860
epoch 609, loss 0.4302, train acc 0.811, test acc 0.860
epoch 610, loss 0.4361, train acc 0.804, test acc 0.860
epoch 611, loss 0.4429, train acc 0.808, test ac

epoch 741, loss 0.4343, train acc 0.807, test acc 0.860
epoch 742, loss 0.4320, train acc 0.808, test acc 0.860
epoch 743, loss 0.4387, train acc 0.804, test acc 0.861
epoch 744, loss 0.4309, train acc 0.811, test acc 0.860
epoch 745, loss 0.4342, train acc 0.808, test acc 0.860
epoch 746, loss 0.4296, train acc 0.808, test acc 0.861
epoch 747, loss 0.4302, train acc 0.815, test acc 0.860
epoch 748, loss 0.4319, train acc 0.807, test acc 0.860
epoch 749, loss 0.4303, train acc 0.812, test acc 0.860
epoch 750, loss 0.4329, train acc 0.811, test acc 0.860
epoch 751, loss 0.4331, train acc 0.814, test acc 0.861
epoch 752, loss 0.4319, train acc 0.812, test acc 0.860
epoch 753, loss 0.4296, train acc 0.812, test acc 0.860
epoch 754, loss 0.4330, train acc 0.812, test acc 0.867
epoch 755, loss 0.4380, train acc 0.807, test acc 0.860
epoch 756, loss 0.4332, train acc 0.810, test acc 0.860
epoch 757, loss 0.4463, train acc 0.800, test acc 0.861
epoch 758, loss 0.4328, train acc 0.810, test ac

epoch 888, loss 0.4337, train acc 0.807, test acc 0.861
epoch 889, loss 0.4385, train acc 0.801, test acc 0.859
epoch 890, loss 0.4407, train acc 0.808, test acc 0.860
epoch 891, loss 0.4336, train acc 0.814, test acc 0.860
epoch 892, loss 0.4323, train acc 0.815, test acc 0.860
epoch 893, loss 0.4579, train acc 0.797, test acc 0.859
epoch 894, loss 0.4316, train acc 0.807, test acc 0.859
epoch 895, loss 0.4298, train acc 0.812, test acc 0.860
epoch 896, loss 0.4310, train acc 0.808, test acc 0.861
epoch 897, loss 0.4312, train acc 0.812, test acc 0.860
epoch 898, loss 0.4299, train acc 0.811, test acc 0.860
epoch 899, loss 0.4315, train acc 0.808, test acc 0.861
epoch 900, loss 0.4318, train acc 0.808, test acc 0.860
epoch 901, loss 0.4316, train acc 0.810, test acc 0.860
epoch 902, loss 0.4304, train acc 0.811, test acc 0.861
epoch 903, loss 0.4360, train acc 0.806, test acc 0.860
epoch 904, loss 0.4308, train acc 0.811, test acc 0.859
epoch 905, loss 0.4309, train acc 0.810, test ac

In [145]:
sub = nd.array(sub)

In [148]:
net(sub).argmax(axis=1).asnumpy()

array([ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,
        0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,
        1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,
        1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0

In [152]:
submission = pd.DataFrame({"PassengerId":pd.read_csv("test.csv")["PassengerId"],
                          "Survived":net(sub).argmax(axis=1).asnumpy().astype("int")
                          })
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [153]:
submission.to_csv("submission_mxnet.csv",index=False)